In [33]:
import csv
from datetime import datetime
import pytz

class Entry:
    def __init__(self, date, time, amount, description, entry_type):
        self.date = date
        self.time = time
        self.amount = amount
        self.description = description
        self.entry_type = entry_type

    def to_list(self):
        return [self.date, self.time, self.amount, self.description, self.entry_type]

class DailyRecord:
    def __init__(self, date):
        self.date = date
        self.incomes = []
        self.expenses = []

    def add_income(self, amount, description):
        time = self.get_current_time_in_thailand()
        income = Entry(self.date, time, amount, description, 'income')
        self.incomes.append(income)
    
    def add_expense(self, amount, description):
        time = self.get_current_time_in_thailand()
        expense = Entry(self.date, time, amount, description, 'expense')
        self.expenses.append(expense)
    
    def get_summary(self):
        total_income = sum(income.amount for income in self.incomes)
        total_expense = sum(expense.amount for expense in self.expenses)
        balance = total_income - total_expense
        return {
            "date": self.date,
            "total_income": total_income,
            "total_expense": total_expense,
            "balance": balance
        }

    def to_list(self):
        return [entry.to_list() for entry in self.incomes + self.expenses]

    @staticmethod
    def get_current_time_in_thailand():
        utc_now = datetime.now(pytz.utc)
        thailand_tz = pytz.timezone('Asia/Bangkok')
        thailand_now = utc_now.astimezone(thailand_tz)
        return thailand_now.strftime('%H:%M:%S')

class Ledger:
    def __init__(self, csv_file):
        self.records = {}
        self.csv_file = csv_file
        self.load_from_csv()
    
    def add_income(self, amount, description):
        date = datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d')
        if date not in self.records:
            self.records[date] = DailyRecord(date)
        self.records[date].add_income(amount, description)
        self.save_to_csv()
    
    def add_expense(self, amount, description):
        date = datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d')
        if date not in self.records:
            self.records[date] = DailyRecord(date)
        self.records[date].add_expense(amount, description)
        self.save_to_csv()
    
    def get_daily_summary(self, date):
        if date in self.records:
            return self.records[date].get_summary()
        else:
            return None

    def get_all_summaries(self):
        return [record.get_summary() for record in self.records.values()]

    def save_to_csv(self):
        with open(self.csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['date', 'time', 'amount', 'description', 'type', 'cumulative_income', 'cumulative_expense', 'running_balance'])
            
            cumulative_income = 0
            cumulative_expense = 0
            for record in self.records.values():
                for entry in sorted(record.incomes + record.expenses, key=lambda e: e.time):
                    if entry.entry_type == 'income':
                        cumulative_income += entry.amount
                    elif entry.entry_type == 'expense':
                        cumulative_expense += entry.amount
                    running_balance = cumulative_income - cumulative_expense
                    writer.writerow([entry.date, entry.time, entry.amount, entry.description, entry.entry_type, cumulative_income, cumulative_expense, running_balance])

    def load_from_csv(self):
        try:
            with open(self.csv_file, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    date, time, amount, description, entry_type, cumulative_income, cumulative_expense, running_balance = row
                    amount = float(amount)
                    cumulative_income = float(cumulative_income)
                    cumulative_expense = float(cumulative_expense)
                    running_balance = float(running_balance)
                    
                    if date not in self.records:
                        self.records[date] = DailyRecord(date)
                    
                    entry = Entry(date, time, amount, description, entry_type)
                    if entry_type == 'income':
                        self.records[date].incomes.append(entry)
                    elif entry_type == 'expense':
                        self.records[date].expenses.append(entry)
        except FileNotFoundError:
            # No CSV file found, nothing to load
            pass

In [34]:
# Example usage
ledger = Ledger('financial_records.csv')
# ledger.add_income(3438, 'Saveing')
ledger.add_expense(2000, 'Friend Rent')
# ledger.add_expense(106, 'Diner')

summary = ledger.get_daily_summary(datetime.now().strftime('%Y-%m-%d'))
print(summary)

all_summaries = ledger.get_all_summaries()
print(all_summaries)

{'date': '2024-07-20', 'total_income': 0, 'total_expense': 2106.0, 'balance': -2106.0}
[{'date': '2024-07-19', 'total_income': 3438.0, 'total_expense': 130.0, 'balance': 3308.0}, {'date': '2024-07-20', 'total_income': 0, 'total_expense': 2106.0, 'balance': -2106.0}]


In [35]:
import pandas as pd
df = pd.read_csv('financial_records.csv')
df

,date,time,amount,description,type,cumulative_income,cumulative_expense,running_balance
0,2024-07-19,05:53:31,3438.0,Saveing,income,3438.0,0.0,3438.0
1,2024-07-19,05:53:31,65.0,Lunch,expense,3438.0,65.0,3373.0
2,2024-07-19,05:53:31,65.0,Diner,expense,3438.0,130.0,3308.0
3,2024-07-20,08:30:36,106.0,Lunch,expense,3438.0,236.0,3202.0
4,2024-07-20,15:53:27,2000.0,Friend Rent,expense,3438.0,2236.0,1202.0
